In [3]:
import pandas as pd 
import numpy as np
import re
import os
import argparse
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SimpleRNN , Dropout ,LSTM,Conv1D, GlobalMaxPooling1D, MaxPooling1D , SpatialDropout1D,Activation,Dense, Dropout, Activation, Flatten, Input, concatenate , GRU
from keras.layers.embeddings import Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
#import xgboost as xgb

In [4]:
stop_words = stopwords.words('arabic')

In [5]:
dataset1 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\dataarabTOx.csv')
dataset6 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\Book1.csv')
new = dataset6.dropna()
# print(new.iloc[10])
# print(new.iloc[9])
dataset7 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\Book2.csv')
dataset = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\toxic arabic tweets classification - Copy.tsv' , delimiter = '\t' , quoting = 3)
dataset2 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-dev.tsv' , delimiter = '\t' , quoting = 3)
dataset3 = pd.read_csv('C:\\Users\\Mahmoud\\Desktop\\OSACT2020-sharedTask-train.tsv' , delimiter = '\t' , quoting = 3)
dataset4 =dataset2.iloc[:,0:2]
dataset5 =dataset3.iloc[:,0:2]
df_concat = pd.concat([dataset,dataset5,dataset4])
for i in range(0,len(df_concat)):
    if df_concat['Class'].iloc[i] == 'normal' or df_concat['Class'].iloc[i] == 'NOT_OFF' or df_concat['Class'].iloc[i] == 'Non-Offensive' or df_concat['Class'].iloc[i] == 0:
        df_concat['Class'].iloc[i] = 'N'
    elif df_concat['Class'].iloc[i] == 'abusive' or df_concat['Class'].iloc[i] == 'hate' or df_concat['Class'].iloc[i] == 'OFF' or df_concat['Class'].iloc[i] == 'Offensive' or df_concat['Class'].iloc[i] == -1 or df_concat['Class'].iloc[i] == -2:
        df_concat['Class'].iloc[i] = 'P'
print(df_concat.shape)

(15998, 2)


In [6]:
X = df_concat.iloc[: , 0].values
y = df_concat.iloc[: , -1].values

In [7]:
labelencoder_y=LabelEncoder()
y = labelencoder_y.fit_transform(y)
dataset["label"] = labelencoder_y.fit_transform(dataset["Class"])
for i in range(0,len(y)):
    if y[i]==0 or y[i]==1:
        continue
    else:
        y[i]=0
print(max(y))
z = 0
p =0
for i in range(0,len(y)):
    if y[i]==0:
        z=z+1
    elif y[i]==1:
        p=p+1
print(z,' ',p)

1
10389   5609


In [48]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42,shuffle = True)

In [35]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\u2066"
        u"\u2069"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(data))

def to_lowercase(text):
    return text.lower()

def remove_eng(text):
    return re.sub(r'[A-Za-z]','',str(text))
    
import string
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):

    return [word for word in words if word not in stop_words]

from nltk.stem.isri import ISRIStemmer
def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = ISRIStemmer()
    return ' '.join([lemmatizer.stem(word) for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_emojis(text)
    text = remove_eng(text)
    text = remove_punctuation(text)
    text = remove_punctuations(text)
    text = normalize_arabic(text)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    words = lemmatize_words(words)
    return ''.join(words)

def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [51]:
X_train = normalize_corpus(X_train)
X_test = normalize_corpus(X_test)

In [50]:
normalize_corpus(["يا مهبلني ومهبلهم يا مجنني ومجننهم يا ليوو"])

['هبل هبل جني ومج ليو']

In [52]:
X_train

['هبل هبل جني ومج ليو',
 'شعب سود شرب كسر قرن',
 'ليك زمك عرف حلك شو عم تحك مين انت سسا انك زلم كنت تحط اسم مش اسم عار لبن هبل انت فا فرق برح طيب حسنل حثل جمع',
 'ودي عرف هالعلب الي طلبو لعو صعب صفه نيه ويه قري نشط دنبوش',
 'حمر',
 'ماتشالاسيوطيبجمهور اما اتحادالكورهالمشبوه قطع ام فرق وهت بقي شوه جماهير الي قعد اكن',
 'دعل انت دان عذر شدد شعب خلج عمه شعب لكه امر شكل خصم اين اتت',
 'صبر هنك كسم يبن احب',
 'شكي زوج حق شيخ زوج وطل لسن نصح مدح بكل وقت جبت غدء اخذ لقم وقل اله عمر قلب روح اكل عصب زوج زبد قلت سنن امدح اكل يوم غدء جرت دحت وتر بصح اكل نحس نحس بسم',
 'قمر نجم شهد معك منك',
 'شرموط عمل حاج غرب انت متستحملوهاش',
 'وزر خرج بنا جبر اسل عتلو حمي يقل عمد قره شرق سلي كسر حصر علي قرت حضر قمه',
 'نعل دين امك نعل دين اهل نعل دين الي جبك',
 'غير بدو هدل وسخ',
 'عنا جواسيس وزر',
 'لول مره تفق معو لنو كلمو صح ميه ميه',
 'شو هالدول نحن عيش عقل احد قدر خرس او شلح سجن قتل سرق طلب سجن لغر شو هالوقاحه عند',
 'الي يقل يبي سيل مطر انه حمر غبي جهل اذكر',
 'قلن يعن لاء حرزو خلي عم هوشو نسل فيه زهق بل

In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,3), tokenizer=text2words,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [54]:
X_train = vec.fit_transform(X_train)
X_test = vec.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier(n_estimators = 2000)
rand.fit(X_train,y_train)

RandomForestClassifier(n_estimators=2000)

In [13]:
from sklearn.metrics import accuracy_score
y_pred = rand.predict(X_test)
score = accuracy_score(y_test,y_pred)
print(score*100)

74.40625


In [55]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
classifier1 = SVC(kernel = 'rbf',C = 10000,probability=False)
classifier1.fit(X_train , y_train)

y_pred1 = classifier1.predict(X_test)
score = accuracy_score(y_test,y_pred1)
print(score*100)

89.0


In [56]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test , y_pred1)
cm

array([[2003,   85],
       [ 267,  845]], dtype=int64)

In [68]:
import keras
while True:
        
        inp = input("User: ")
        
        if inp.lower() == "quit":
            break
        else:
            inp = normalize_corpus([inp])
            #print(inp)
            if (len(inp[0]) != 0):
                result = classifier1.predict(vec.transform(inp))
                print(result)
                if result == 0:
                    print("normal")
                else:
                    print("abusive")
            else:
                result = 0
                print(result)
                print("normal")

User: 
0
normal
User: dhugr
0
normal
User: ياعلق
[1]
abusive
User: علق الاستاذ ع هذه القصه فاجاب اجابه رشيده
[0]
normal
User: Quit


In [64]:
len("")

0